Voorbeeld implementatie van het bevragen van het CROW LDP endpoint met Python

Gebruikt python 3 met hashlib en hmac (met dank aan https://www.jokecamp.com/blog/examples-of-creating-base64-hashes-using-hmac-sha256-in-different-languages/ )

Let op: de key en value waarden die de hmac in gaan staan precies andersom dan in de javascript implementatie !!!

installeer (mini)conda, maak een nieuwe environment. installeer jupyter en requests. start jupyter notebook.


In [ ]:
from os.path import expanduser
import json

#Laad parameters van prive configuratiefile
connection_data=json.load(open(expanduser("~/Stuff/config.json")))
clientId = connection_data["clientId"]
toolId = connection_data["toolId"]
privateKey = connection_data["privateKey"]
base_url = connection_data["base_url"]

#config.json:
#{
#    "clientId":"<clientId>",
#    "toolId":"<toolId>",
#    "privateKey":"<privateKey>",
#    "base_url":"<base_url>"
#}

# #of prompt er voor:
# import getpass
# privateKey = getpass.getpass("Please enter privateKey: ")
# clientId = ""
# toolId = ""
# base_url = ""

In [ ]:
import datetime
import uuid
import hashlib
import hmac
import base64

#Functie om de HMAC signature te berekenen, zie documentatie van CROW voor de benodigde parameters
def get_hmac(req):
    #Timestamp (op secondes, afgesloten met een 'Z')
    currentDate = datetime.datetime.now().replace(microsecond=0).isoformat()+"Z"
    #Random GUID conform RFC 4122 
    nonce = str(uuid.uuid4()) 
    
    #Afhankelijk van een GET of POST request is het signature anders    
    method=req.method
    message = method+","+ currentDate + "," + url + "," + nonce
    
    if method == "POST":
        contentType = req.headers['Content-Type']
        body = req.data
        if len(body)>0:
            md5 = hashlib.md5(req.data.encode('utf-8')).hexdigest()
            message += "," + contentType + "," + md5
    
    #print(message)
    value = bytes(message, 'utf-8')
    #privateKey wordt eerder in het notebook opgehaald, net als clientId
    privkey = bytes(privateKey, 'utf-8')
    
    hashBase64 = base64.b64encode(hmac.new(privkey, value, digestmod=hashlib.sha256).digest())
    authorization = "HMAC clientId=\"" + clientId + "\", nonce=\"" + nonce + "\", currentDate=\"" + currentDate + "\", signature=\"" + hashBase64.decode("utf-8")  + "\""
    
    return authorization

In [ ]:
## GET Request
## omdat de URL precies hetzelfde moet zijn voor de hmac berekening als 
## in het request wordt de url hier samengesteld ipv te werken met de request parameters...

import requests
import urllib.parse

s = Session()
url = base_url
querystring = {"output":"csv",
               "toolid":toolId,
               "trace":"namespaces"}

payload = "select * { ?s ?p ?o . } limit 10"

url = str(url)+ "?query=" + \
        urllib.parse.quote(payload) + "&" + \
        urllib.parse.urlencode(querystring) 

#het request wordt hier gedeeltelijk samengesteld, zodat de HMAC signature berekend kan worden
req = requests.Request("GET", url)
headers = {
    'Authorization': get_hmac(req)
    }
#aanvullend de header in het request zetten
req.headers = headers
prepared=req.prepare()

response = s.send(prepared)

print(response.text)

In [ ]:
## POST Request 
import requests
import urllib.parse

s = Session()
url = base_url
querystring = {"output":"json",
               "toolid":toolId,
               "trace":"namespaces"}

payload = "select * { ?s ?p ?o . } limit 10"
url = str(url)+ "?" + \
        urllib.parse.urlencode(querystring) 

headers = {
    'Content-Type': "application/sparql-query"
    }
#het request wordt hier gedeeltelijk samengesteld, zodat de HMAC signature berekend kan worden
req = requests.Request("POST", url, data=payload, headers=headers)

auth = get_hmac(req)
headers = {
    'Authorization': auth ,
    'Content-Type': "application/sparql-query"
    }
#de header van het request opnieuw zetten, nu met de HMAC signature
req.headers = headers
prepared=req.prepare()
response = s.send(prepared)

print(json.dumps(response.json(), indent=2, sort_keys=True)) 

In [ ]:
#helper functie om het request te doen, zodat verderop makkelijker gewerkt kan worden met verschillende queries

from requests import Request, Session
import urllib.parse

def run_query(payload): 
    s = Session()
    url = base_url
    querystring = {"output":"json",
               "toolid":toolId,
               "trace":"namespaces"}
    url = str(url)+ "?" + \
        urllib.parse.urlencode(querystring) 

    req = requests.Request("POST", url, data=payload)
    headers = {'Content-Type': "application/sparql-query"}
    req.headers = headers
    auth = get_hmac(req)
    headers = {
        'Authorization': auth ,
        'Content-Type': "application/sparql-query"
        }
    req.headers = headers
    prepared=req.prepare()
    response = s.send(prepared)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(response.status_code, response.text))

In [ ]:
#Met bovenstaande helperfuncties wordt het nu overzichtelijk om queries te gaan uitvoeren...
import json
from pandas.io.json import json_normalize
import pandas as pd

#pandas opmaak zetten
pd.options.display.width=120
pd.options.display.max_colwidth=100

#selecteer alle typen de gebruikt worden:
query = """Select  distinct   ?o
    where { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?o .
    }
    ORDER BY STR(?o)
    limit 1000
    """
result = run_query(query) 

#json_normalize maakt een pandas dataframe / tabel van het resultaat
json_normalize(result['results']['bindings'])


In [ ]:
#specificaties aan objecten:
query = """Select  distinct   ?Objecten ?EisId ?Specificatie
where { ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontologie.crow.nl/bibliotheekspecificatie/201711/Object>.
        ?s <http://www.w3.org/2004/02/skos/core#prefLabel> ?Objecten.
        ?s <http://ontologie.crow.nl/bibliotheekspecificatie/201711/isGespecificeerdDoor> ?y.
        ?y <http://www.crowprocontractdata.nl/ns/topteam/2016/11/21/schema/id> ?EisId.
        ?y <http://ontologie.crow.nl/bibliotheekspecificatie/201711/beschrijving> ?Specificatie.
      } 
 ORDER BY STR(?Objecten)
limit 100"""
result = run_query(query) 

#json_normalize maakt een pandas dataframe / tabel van het resultaat
json_normalize(result['results']['bindings'])

In [ ]:
query="""select   distinct  ?Publicatie  ?Hoofdstuk ?Deelhoofdstuk ?Paragraaf ?Artikel ?Lid ?LidTekst ?JC_naam ?Fac ?Objecten ?Activiteiten ?Funkties  ?Raakvlak ?VerificatieMethode ?EisType

where {?s  <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontologie.crow.nl/document/201711/Publicatie>.
       ?s <http://www.w3.org/2004/02/skos/core#prefLabel> ?Publicatie .
       filter (STR(?Publicatie)="CROW Specificaties 2018 Standaard en Aanvullingen RAW [RDOC-321209]").
 
       ?s <http://ontologie.crow.nl/document/201711/bevat> ?y.
       ?y <http://www.w3.org/2004/02/skos/core#prefLabel> ?Hoofdstuk.
        
       ?y <http://ontologie.crow.nl/document/201711/bevat> ?z.
       ?z <http://www.w3.org/2004/02/skos/core#prefLabel> ?Deelhoofdstuk.
       
       ?z <http://ontologie.crow.nl/document/201711/bevat> ?x.
       ?x <http://www.w3.org/2004/02/skos/core#prefLabel> ?Paragraaf.
       
       ?x <http://ontologie.crow.nl/document/201711/bevat> ?q.
       ?q <http://www.w3.org/2004/02/skos/core#prefLabel> ?Artikel.
       
       ?q <http://ontologie.crow.nl/document/201711/bevat> ?r.
       
#       ?r ?p ?o .
       ?r <http://ontologie.crow.nl/bibliotheekspecificatie/201711/naam> ?Lid.
       ?r <http://ontologie.crow.nl/bibliotheekspecificatie/201711/beschrijving> ?LidTekst.
     # aangeven of de specificatie UAV of UAVgc is of niet
       Optional{
         ?r <http://ontologie.crow.nl/document/201711/isVastBinnenJuridischeContext> ?JContext.
         ?JContext <http://www.w3.org/2004/02/skos/core#prefLabel> ?JC_naam.
       filter( STR(?JC_naam) = "Context UAV").
 #        filter( STR(?JC_naam)="Context UAV-GC").
       }
    # zijn er ook specificties die facultatief zijn binnen de UAV context
       Optional{
         ?r <http://ontologie.crow.nl/document/201711/isFacultatiefBinnenJuridischeContext>  ?Yfac.
         ?Yfac <http://www.w3.org/2004/02/skos/core#prefLabel> ?Fac.
        filter( STR(?Fac)="Context UAV").
        }
     # de gekoppelde objecten ophalen aan de specificatie
        Optional{
           ?r <http://ontologie.crow.nl/bibliotheekspecificatie/201711/specificeert> ?y1.
           ?y1 <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontologie.crow.nl/bibliotheekspecificatie/201711/Object> .
           ?y1 <http://www.w3.org/2004/02/skos/core#prefLabel> ?Objecten.
         }
     # de gekoppelde activiteiten ophalen aan de specificatie
     
        Optional{
           ?r <http://ontologie.crow.nl/bibliotheekspecificatie/201711/specificeert> ?y2.
           ?y2 <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontologie.crow.nl/bibliotheekspecificatie/201711/Activiteit> .
           ?y2 <http://www.w3.org/2004/02/skos/core#prefLabel> ?Activiteiten.
         }
     # de gekoppelde funktie ophalen aan de specificatie
     
        Optional{
           ?r <http://ontologie.crow.nl/bibliotheekspecificatie/201711/specificeert> ?y5.
           ?y5 <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontologie.crow.nl/bibliotheekspecificatie/201711/Functie> .
           ?y5 <http://www.w3.org/2004/02/skos/core#prefLabel> ?Funkties.
         } 
     # de gekoppelde raakvlak ophalen aan de specificatie
     
        Optional{
           ?r <http://ontologie.crow.nl/bibliotheekspecificatie/201711/specificeert> ?y6.
           ?y6 <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontologie.crow.nl/bibliotheekspecificatie/201711/Raakvlak> .
           ?y6 <http://www.w3.org/2004/02/skos/core#prefLabel> ?Raakvlak.
         }     
     # de gekoppelde verificatiemethode
        Optional{
           ?r <http://ontologie.crow.nl/bibliotheekspecificatie/201711/heeftVerificatiemethode> ?y3.
           ?y3 <http://www.w3.org/2004/02/skos/core#prefLabel> ?VerificatieMethode.
         }
     # type van de eis ophalen
        Optional{
           ?r <http://ontologie.crow.nl/bibliotheekspecificatie/201711/isVanCategorie> ?y4.
           ?y4 <http://www.w3.org/2004/02/skos/core#prefLabel> ?EisType.
         }
       
        }

ORDER BY str(?Hoofdstuk)  str(?Deelhoofdstuk) str(?Paragraaf) str(?Artikel) str(?Lid)


limit 100
"""
result = run_query(query) 

#json_normalize maakt een pandas dataframe / tabel van het resultaat
json_normalize(result['results']['bindings'])